## Archive

This notebook looks at a particular historical fire and generates all the output.

In [ ]:
# If you haven't installed the fireatlas code yet, uncomment the following line and run this cell.

# !pip install -e .. -q

# After this runs, restart the notebook kernel.

In [ ]:
import datetime
import pandas as pd
import geopandas as gpd

from fireatlas import FireMain, FireTime, FireObj, postprocess, preprocess
from fireatlas.utils import timed

tst = [2020, 9, 5, "AM"]
ted = [2020, 9, 8, "PM"]
region = ("Creek", [-119.5, 36.8, -118.9, 37.7])

list_of_ts = list(FireTime.t_generator(tst, ted))

## Preprocess the region

Preprocess the region to get rid of static flare sources. Save that new "swiss cheese" shape off into a geojson file for later.

In [ ]:
preprocess.preprocess_region(region)

## Are preprocessed input files available?

Check to see if all the files that we need have already been processed and are available on s3.

Note: "NOAA20" files for this time period are not in the input folder so we will only use "SNPP". Another way to do this
for DPS jobs is to override the `FIRE_SOURCE` constant. You can export an os environment 
variable called `FEDS_FIRE_SOURCE='VIIRS'` or `FEDS_FIRE_SOURCE='SNPP'`

In [ ]:
ts_that_need_preprocessing = preprocess.check_preprocessed_file(tst, ted, sat="SNPP", freq="monthly", location="s3")

In [ ]:
assert len(ts_that_need_preprocessing) == 1, "There is one monthly file that is not preprocessed on s3"

Since there is one file that needs processing, let's process that and then we can read from local.

In [ ]:
for t in ts_that_need_preprocessing:
    preprocess.preprocess_monthly_file(t, "SNPP")

## Preprocess for region and t

Do initial filtering and clustering using the preprocessed region (from local) and the half day files (also from local).

In [ ]:
%%time
for t in list_of_ts:
    preprocess.preprocess_region_t(t, region=region, read_location="local")

## Run the fire expansion and merging algorithm

Now that we have all the region-specific preprocessed fires, we can go ahead and run `Fire_Forward`.

In [ ]:
FireMain.Fire_Forward(tst=tst, ted=ted, region=region, read_location="local")

## Write Snapshot files

Start by reading in the outputs from the run:

In [ ]:
allpixels = postprocess.read_allpixels(tst, ted, region, location="local")
allfires_gdf = postprocess.read_allfires_gdf(tst, ted, region, location="local")

Then iterate through all of the ts and save the snapshot layers.

In [ ]:
%%time
postprocess.save_snapshots(allfires_gdf, region, tst, ted)

## Write Largefire files

First we will find all the large fires:

In [ ]:
large_fires = postprocess.find_largefires(allfires_gdf)

In this case there is only one since we set the region and time specifically to look at th Creek Fire

In [ ]:
large_fires

First we'll use the `allpixels` object to create the `nplist` layer

In [ ]:
%%time
postprocess.save_large_fires_nplist(allpixels, region, large_fires, tst)

The rest of the layers will be created directly from the `allfires_gdf` and have some merge handling

In [ ]:
%%time
postprocess.save_large_fires_layers(allfires_gdf, region, large_fires, tst)